<a href="https://colab.research.google.com/github/Jeniejean/ANALYTICS-AND-TOOLS-AND-PROGRAMMING/blob/main/Analyzing%20Sentiment%20in%20Song%20Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%pip install dash pandas dash_bootstrap_components dash_core_components plotly transformers

In [7]:
import dash
import pandas as pd
import dash_bootstrap_components as dbc
from dash import dcc
import plotly.graph_objects as go
from dash import html, dash_table
from dash.dependencies import Input, Output, State
from transformers import pipeline, MarianMTModel, MarianTokenizer

In [8]:
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
import dash_table
import plotly.graph_objects as go
import pandas as pd
from transformers import pipeline, MarianMTModel, MarianTokenizer

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])

# Navbar
nav = dbc.Nav(
    [
        dbc.NavItem(dbc.NavLink("Overview", active=True, href="#")),
        dbc.NavItem(dbc.NavLink("Features", href="#")),
        dbc.NavItem(dbc.NavLink("Contacts", href="#")),
    ],
    style={'display': 'flex', 'flex-direction': 'row'},
)

# Sidebar
sidebar = html.Div("Sidebar Content")

# Button style
button_style = {'background-color': '#C80000', 'color': 'white', 'border': 'none', 'fontSize': '14px', 'fontWeight': 'bold', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '1.5px 1.5px 1.5px #efefef'}

# Layout
app.layout = dbc.Container([
    html.H1("EmoVerse Analyzer", className='Noto_regular_font', style={'textAlign': 'center', 'marginBottom': '15px', 'fontWeight': 'bold', 'fontSize': '35px'}),
    html.Div(nav),
    dbc.Row([
        dbc.Col(width=8, children=[
            html.Div("Unlocking the Soul of Songs", style={'fontSize': '15px', 'textAlign': 'right', 'marginBottom': '1px'}, className='Noto_regular_font'),
            html.Div("Elevating Content Quality and Audience Connection. Dive deep into the heart of music with our app, analyzing song lyrics to reveal their true sentiment and meaning. Enhance audience comprehension and appreciation while boosting your business prospects effortlessly.", style={'fontSize': '10px', 'textAlign': 'right', 'marginTop': '20px', 'marginBottom': '10px'}, className='Noto_regular_font'),
        ]),
        dbc.Col(width=4, style={'height': '250px', 'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'overflow': 'hidden'}, children=[dbc.Card(
            html.Img(src='/assets/01.jpg', className='card-img-top', style={'width': '100%', 'height': 'auto', 'objectFit': 'cover', 'borderRadius': '10px'}),
            className="rounded"
        )])
    ], style={'marginBottom': '20px'}),

    dbc.Row([
        dbc.Col(width=4, style={'height': '500px', 'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'overflow': 'hidden'}, children=[dbc.Card(
            html.Img(src='/assets/02.jpg', className='card-img-top', style={'width': '100%', 'height': 'auto', 'objectFit': 'cover', 'borderRadius': '10px'}),
            className="rounded"
        )]),
        dbc.Col(width=8, children=[
            dbc.Card([
                dbc.CardHeader("Start Now", style={'textAlign': 'center', 'fontWeight': 'bold', 'fontSize': '15px'}, className='Noto_regular_font'),
                dbc.CardBody([
                    # Add your content here
                    html.Div([
                        dbc.Input(id='lyrics_input', type='text', placeholder='Enter your lyrics', style={'marginBottom': '15px', 'fontSize': '13px'}),
                        dbc.Button("GO", n_clicks=0, id='lyrics_submit_button', style=button_style),
                    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
                ]),
            ], style={'box-shadow': '1.5px 1.5px 1.5px #efefef', 'height': '500px'})
        ])
    ], style={'marginTop': '40px'}),

    dbc.Row([
        dbc.Col(width=5, children=[
            dbc.Card([
                dbc.CardHeader("Translated Lyrics", style={'textAlign': 'center', 'fontWeight': 'bold'}, className='Noto_regular_font'),
                dbc.CardBody([
                    html.Div(id='language_output', style={'whiteSpace': 'pre-line', 'height': '50px', 'padding': '10px'}, className='Noto_regular_font'),
                    html.Div(id='translate_output', style={'whiteSpace': 'pre-line', 'height': '550px', 'padding': '5px'}, className='Noto_regular_font'),
                    html.Div(
                        dcc.Clipboard(target_id="translate_output", title="copy", style={"display": "inline-block", "fontSize": 20, "verticalAlign": "top"}),
                        style={"float": "right"})
                ])
            ], style={'box-shadow': '1.5px 1.5px 1.5px #efefef', 'height': '700px'})
        ]),
        dbc.Col(width=7, children=[
            dbc.Card([
                dbc.CardHeader("Classification Table", style={'textAlign': 'center', 'fontWeight': 'bold'}, className='Noto_regular_font'),
                dash_table.DataTable(
                    data=[],
                    style_table={'overflowX': 'auto'},
                    id='data_table',
                    style_cell={'padding': '5px', 'textAlign': 'center', 'fontSize': '14px', 'fontWeight': 'normal'},
                    style_header={'backgroundColor': 'white', 'fontSize': '15px', 'fontWeight': 'bold', 'textAlign': 'center'},
                    style_cell_conditional=[{'if': {'column_id': c}, 'textAlign': 'left'} for c in ['Text']])
            ], style={'box-shadow': '1.5px 1.5px 1.5px #efefef', 'height': '700px'})
        ])
    ], style={'marginTop': '40px'}),

    dbc.Row([
        dbc.Col(width=6, children=[
            dbc.Card([
                dcc.Graph(figure={}, id='sentiment_chart')
            ], style={'box-shadow': '1.5px 1.5px 1.5px #efefef', 'height': '500px'})
        ]),
        dbc.Col(width=6, children=[
            dbc.Card([
                dcc.Graph(figure={}, id='emotion_chart')
            ], style={'box-shadow': '1.5px 1.5px 1.5px #efefef', 'height': '500px'})
        ])
    ], style={'marginTop': '40px'})

], style={'margin': 'auto', 'padding': '40px', 'backgroundColor': '#fcfcfc'})

# Language translation model
@app.callback(
    [Output('language_output', 'children'),
     Output('translate_output', 'children')],
    [Input('lyrics_submit_button', 'n_clicks')],
    [State('lyrics_input', 'value')]
)
def lang_translation(n_clicks, lyrics):
    if n_clicks and lyrics:
        model_ckpt = "papluca/xlm-roberta-base-language-detection"
        pipe = pipeline("text-classification", model=model_ckpt)
        lang_class_result = pipe(lyrics, top_k=1, truncation=True)[0]['label']

        if lang_class_result == 'th':
            model_name = "Helsinki-NLP/opus-mt-th-en"
            lang_name = "Thai"
        elif lang_class_result == 'zh':
            model_name = "Helsinki-NLP/opus-mt-zh-en"
            lang_name = "Chinese"
        elif lang_class_result == 'ja':
            model_name = "Helsinki-NLP/opus-mt-ja-en"
            lang_name = "Japanese"
        else:
            return "The language of this song is: en", lyrics

        model = MarianMTModel.from_pretrained(model_name)
        tokenizer = MarianTokenizer.from_pretrained(model_name)

        text_parts = lyrics.split()
        translated_parts = []

        for part in text_parts:
            inputs = tokenizer(part, return_tensors="pt", padding=True)
            outputs = model.generate(**inputs)
            translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            translated_parts.append(translated_text)

        translated_text = ' '.join(translated_parts)
        return "The language of this song is: {}".format(lang_name), translated_text

    else:
        return dash.no_update, dash.no_update

# Text sentiment classification model
@app.callback(
    Output('data_table', 'data'),
    Input('translate_output', 'children')
)
def sentiment_classification(text):
    if text:
        words = [i.strip() for i in text.split('.')]
        word_count = {}
        for word in words:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

        df = pd.DataFrame(list(word_count.items()), columns=['Text', 'Count'])

        def sentiment_task(text):
            model_path = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
            sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
            return sentiment_task(text)

        def emotion_task(text):
            classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
            return classifier(text)

        df['Sentiment'] = df['Text'].apply(lambda x: sentiment_task(x)[0]['label'])
        df['Emotion'] = df['Text'].apply(lambda x: emotion_task(x)[0][0]['label'])
        df.drop(df[df['Text'] == ''].index, inplace=True)
        return df.to_dict('records')
    else:
        return []

# Sentiment graph
@app.callback(
    Output('sentiment_chart', 'figure'),
    Input('data_table', 'data')
)
def update_sentiment_chart(data):
    if data:
        df = pd.DataFrame(data)
        sentiment_counts = df.groupby('Sentiment').sum().reset_index()
        sentiment_counts = sentiment_counts[sentiment_counts['Sentiment'] != 'neutral']
        fig = go.Figure(data=[
            go.Pie(
                labels=sentiment_counts['Sentiment'],
                values=sentiment_counts['Count'],
                textinfo='label + percent',
                name='Sentiment'),
        ])
        fig.update_traces(
            hole=0.4,
            hoverinfo=" label + percent + name")
        fig.update_layout(
            title_text="Sentiment Classified Donut Chart",
            annotations=[dict(text='Sentiment', x=0.5, y=0.5, font_size=12, showarrow=False)])
        return fig
    else:
        return {}

# Emotions graph
@app.callback(
    Output('emotion_chart', 'figure'),
    Input('data_table', 'data')
)
def update_emotion_chart(data):
    if data:
        df = pd.DataFrame(data)
        emotion_counts = df.groupby('Emotion').sum().reset_index()
        emotion_counts = emotion_counts[emotion_counts['Emotion'] != 'neutral']
        fig = go.Figure(data=[
            go.Pie(
                labels=emotion_counts['Emotion'],
                values=emotion_counts['Count'],
                textinfo='label + percent',
                name='Emotion'),
        ])
        fig.update_traces(
            hole=0.4,
            hoverinfo=" label + percent + name")
        fig.update_layout(
            title_text="Emotional Classified Donut Chart",
            annotations=[dict(text='Emotional', x=0.5, y=0.5, font_size=12, showarrow=False)])
        return fig
    else:
        return {}

if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>